In [1]:
import tensorflow as tf

In [26]:
true_label = tf.constant([[0,0,1],[0,1,0],[1,0,0]],dtype=tf.float32)
pred_label = tf.constant([[0.6,0.5,1],[0,0,0],[1,0,0]],dtype=tf.float32)

In [27]:
y_pred = tf.round(pred_label)
# y_true_index = tf.where(y_true==1)
# y_pred_index = tf.where(y_pred==1)

In [28]:
true_label.dtype

tf.float32

In [29]:
tf.norm(tensor=y_pred-true_label, ord=1)

<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [40]:
def customized_loss(true_label, predicted_label): # overlap_distance_calculate provided by DARPA
    """
    true_label, predicted_label: 2d tensor, the true_label with 0s and 1s only
    min_valid_range: the maximum distance in % of the largest size of the image (diagonal)
    between a predicted pixel vs. a true one that will be considered
    as valid to include in the scoring.
    calculate_distance: when True this will not only calculate overlapping pixels
        but also the distances between nearesttrue and predicted pixels
    """
    min_valid_range=.1
    
    predicted = tf.cast(predicted_label>0.5, tf.float32) # cutoff = 0.5 to mask the prediction
    
    points_done_pred=[]
    points_done_true=[]
    
    # first calculate the overlapping pixels
    mat_overlap=true_label*predicted
    
    for x_true, y_true in tf.where(mat_overlap==1): 
        points_done_true.append((x_true, y_true))
        points_done_pred.append((y_true, x_true))
        
    # find all pixels that are true but not predicted 
    candidates = [(x_true, y_true) for x_true, y_true in tf.where(true_label==1) if (x_true, y_true) not in points_done_true]
    
    diagonal_length = tf.sqrt(tf.pow(true_label.shape[0],2)+tf.pow(true_label.shape[1],2))
    min_valid_range=tf.cast((min_valid_range*diagonal_length)/100, tf.int) # in pixels
    
    def nearest_pixels(x_true, y_true):
        result=tf.constant(0.0)
        # find all the points in pred withing min_valid_range rectangle
        mat_pred_inrange=predicted[
            max(x_true-min_valid_range, 0): min(x_true+min_valid_range, mat_overlap.shape[1]),
            max(y_true-min_valid_range, 0): min(y_true+min_valid_range, mat_overlap.shape[0])
        ]
        if (tf.where(mat_pred_inrange==1).shape[0] == 0): return 100 # no predicted in the nearby region
        for x_pred_shift, y_pred_shift in tf.where(mat_pred_inrange==1): 
            y_pred=max(y_true-min_valid_range, 0)+y_pred_shift
            x_pred=max(x_true-min_valid_range, 0)+x_pred_shift
            if (x_pred, y_pred) in points_done_pred:
                continue
            # calculate eucledean distances 
            dist_square=tf.pow(x_true-x_pred, 2)+tf.pow(y_true-y_pred, 2)
            result+=dist_square
        return result

    loss = tf.reduce_sum([nearest_pixels(x_true, y_true) for x_true, y_true in candidates])
    return loss
    
    

In [41]:
A = tf.constant(1.0)
B = tf.constant(2.0)
C = mi(A,1.5)
C

1.5

In [42]:
true_label = tf.constant([[0,0,1],[0,1,0],[1,0,0]],dtype=tf.float32)
pred_label = tf.constant([[0.0,0.0,1.0],[0.0,0.0,0.0],[1.0,0.0,0.0]])
mat_overlap = true_label*pred_label
summ = tf.reduce_sum(true_label)
summ

<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [45]:
pred_label[0][2]

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [43]:
A = customized_loss(true_label,pred_label)

NotFoundError: Could not find valid device for node.
Node:{{node Sqrt}}
All kernels registered for op Sqrt :
  device='XLA_GPU'; T in [DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_BFLOAT16, DT_COMPLEX128, DT_HALF]
  device='XLA_CPU'; T in [DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_BFLOAT16, DT_COMPLEX128, DT_HALF]
  device='XLA_CPU_JIT'; T in [DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_BFLOAT16, DT_COMPLEX128, DT_HALF]
  device='XLA_GPU_JIT'; T in [DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_BFLOAT16, DT_COMPLEX128, DT_HALF]
  device='GPU'; T in [DT_DOUBLE]
  device='GPU'; T in [DT_HALF]
  device='GPU'; T in [DT_FLOAT]
  device='CPU'; T in [DT_COMPLEX128]
  device='CPU'; T in [DT_COMPLEX64]
  device='CPU'; T in [DT_BFLOAT16]
  device='CPU'; T in [DT_DOUBLE]
  device='CPU'; T in [DT_HALF]
  device='CPU'; T in [DT_FLOAT]
 [Op:Sqrt]

In [32]:
tf.cast(pred_label>0.5, tf.float32)

TypeError: Cannot convert 0.5 to EagerTensor of dtype int32

In [10]:
mat_overlap

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[0, 0, 1],
       [0, 0, 0],
       [1, 0, 0]], dtype=int32)>

In [27]:
tf.where(mat_overlap==1).shape[0]

2

In [28]:
result=tf.constant(0.0)

In [29]:
result

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>